# --- Day 11: Seating System ---

Your plane lands with plenty of time to spare. The final leg of your journey is a ferry that goes directly to the tropical island where you can finally start your vacation. As you reach the waiting area to board the ferry, you realize you're so early, nobody else has even arrived yet!

By modeling the process people use to choose (or abandon) their seat in the waiting area, you're pretty sure you can predict the best place to sit. You make a quick map of the seat layout (your puzzle input).

The seat layout fits neatly on a grid. Each position is either floor (.), an empty seat (L), or an occupied seat (#). For example, the initial seat layout might look like this:

    L.LL.LL.LL
    LLLLLLL.LL
    L.L.L..L..
    LLLL.LL.LL
    L.LL.LL.LL
    L.LLLLL.LL
    ..L.L.....
    LLLLLLLLLL
    L.LLLLLL.L
    L.LLLLL.LL

Now, you just need to model the people who will be arriving shortly. Fortunately, people are entirely predictable and always follow a simple set of rules. All decisions are based on the number of occupied seats adjacent to a given seat (one of the eight positions immediately up, down, left, right, or diagonal from the seat). The following rules are applied to every seat simultaneously:

    If a seat is empty (L) and there are no occupied seats adjacent to it, the seat becomes occupied.
    If a seat is occupied (#) and four or more seats adjacent to it are also occupied, the seat becomes empty.
    Otherwise, the seat's state does not change.

Floor (.) never changes; seats don't move, and nobody sits on the floor.

After one round of these rules, every seat in the example layout becomes occupied:

    #.##.##.##
    #######.##
    #.#.#..#..
    ####.##.##
    #.##.##.##
    #.#####.##
    ..#.#.....
    ##########
    #.######.#
    #.#####.##

After a second round, the seats with four or more occupied adjacent seats become empty again:

    #.LL.L#.##
    #LLLLLL.L#
    L.L.L..L..
    #LLL.LL.L#
    #.LL.LL.LL
    #.LLLL#.##
    ..L.L.....
    #LLLLLLLL#
    #.LLLLLL.L
    #.#LLLL.##

This process continues for three more rounds:

    #.##.L#.##
    #L###LL.L#
    L.#.#..#..
    #L##.##.L#
    #.##.LL.LL
    #.###L#.##
    ..#.#.....
    #L######L#
    #.LL###L.L
    #.#L###.##

    #.#L.L#.##
    #LLL#LL.L#
    L.L.L..#..
    #LLL.##.L#
    #.LL.LL.LL
    #.LL#L#.##
    ..L.L.....
    #L#LLLL#L#
    #.LLLLLL.L
    #.#L#L#.##

    #.#L.L#.##
    #LLL#LL.L#
    L.#.L..#..
    #L##.##.L#
    #.#L.LL.LL
    #.#L#L#.##
    ..L.L.....
    #L#L##L#L#
    #.LLLLLL.L
    #.#L#L#.##

At this point, something interesting happens: the chaos stabilizes and further applications of these rules cause no seats to change state! Once people stop moving around, you count 37 occupied seats.

Simulate your seating area by applying the seating rules repeatedly until no seats change state. How many seats end up occupied?


In [60]:
import numpy as np

with open("day11_input.txt") as f:
    seats = f.readlines()
seats_current = [line.strip() for line in seats]

def count_occupied_adjacent(row, col, seats, show_used=False):
    count = 0
    row_min = row-1 if row-1 >= 0 else row
    row_max = row+2 if row+1 <= len(seats) else row+1
    col_min = col-1 if col-1 >= 0 else col
    col_max = col+2 if col+1 <= len(seats[0]) else col+1
    
    seat_count = 0
    for seat_rows in seats[row_min:row_max]:
#         print(seat_rows)
        for seat in seat_rows[col_min:col_max]:
            if show_used == True:
                print(seat)
            if seat == "#":  # Occupied
                seat_count += 1
    if seats[row][col] == '#':
        seat_count -= 1
#     print(seat_count)
    return seat_count

moving = True
while moving:
    seats_future = []
    for row_num, seat_rows in enumerate(seats_current):
        new_seat_row = ''
        for col_num, seat in enumerate(seat_rows):
            if seat == "#" or seat == "L":
                num_occupied = count_occupied_adjacent(row_num, col_num, seats_current)
                
                if seat == "L":
                    if num_occupied == 0:
                        new_seat_row += "#"
                    else:
                        new_seat_row += seat
                if seat == "#":
                    if num_occupied >= 4:
                        new_seat_row += "L"
                    else:
                        new_seat_row += seat

            else:
                new_seat_row += seat
        seats_future.append(new_seat_row)
    
#     zero_two = count_occupied_adjacent(2, 0, seats_current, show_used=True)
#     print(f"zero, two: {zero_two}")
    
    [print(row) for row in seats_current]
    print()
    if seats_future == seats_current:
        moving = False
    else:
        seats_current = seats_future.copy()

full_count = 0
for seat_rows in seats_current:
    for seat in seat_rows:
        if seat == "#":
            full_count += 1
print(full_count)

LLLLL.LLLLLLL.LLLLLL.L.LLLL..LLLL.LLLLLLLLL.LLLLLLLL.LLLLLLLLLLLLL.L.LLLLLLLLLLLLLL.LLLLLL
LLLLLLLLLLLLLLL.LLLL..LLLLLLLLLLLL.LLLLLLLL.L..LLLLLLLLLLLL.LLLL.LLL.LLLLL.LLL.LLLLLLLLLLL
LLLLL.LLLLLLLLLLLLLLLLLLLLL.LLLLLL.LLLLLLLLLLL.LLL.LLLLLLLL.LLLLL.LL.LLLLL.LLLLLLLL.LLLLLL
LLLLL.LLLLLLLLL.LLLL.LLLLLL.LLLLLL.LLLLLLLL.LLLLLLLL.LLLLLLLLLLLLLLLLL.LLL.LLLLLLLL.LLLLLL
LLLLLLLLLLLLLLL.LLLLLLLLLLLLLLLLLL.LLLLLLLL.LLLLLLLL.LLLLLL.LLLLLLLLL.LLLLLLLLLLLLL.LLLLLL
LLLLLLLLLLLLLLL.LLLL.LLLLLLLLLLLLLLLLLLLLLL.L.LLLLLL.LLLLLL..LLLLLL..LLLLLLLLLLLLLL.LLLLLL
.L..L.L.L.LL..LL.........L....L.L..LL.LLLL.L..L.LLLL.L.L..L.......LL...LL..LL.L..L.L.LL..L
LLL.L.LLL.LLLLL..LLL.LLLLLL.LL.L.L.LLLLLLLL.LLLLLLLL.LLLLLL.LLLLLLLL.LLLLL.LLLLLLLL..LLLLL
LL.L..LLLLLLLLLLLLLL.LLLLLL.LLLLLL.LLLLLLLLLLLLLLLLL.LLLLLL.LLLL.LLLLLLLLL.LLLLLLLLLLLLLLL
LLL.LLL.LLLL.LL.LLLL.LLLLLLLLLLLLL.LLLLLLLLLLLLLLLLL.LLLLLLLLLLLLLLL.LLLLLLL.LLLLLLLLLLLLL
LLLLLLLLL.LLLLL.LLLLLLLLLLL.LLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL.LLLLLLLLLLLLLL.LLLLL.LL.LLL

LLLL#.#########.####.LLLL##################.########.LLLL##.##############.LLLL####.LL.LLL
##L#L.L#############.#L#L##.######.########.########.#L#L##.########.#####.#L#L####..#L#L#
.L.L.#..L.L...######.....##...#..L..L......####.#.##.....####.##.#.....#.#...L....#...L...
#L##..L#L#L#.##.###########L#L##L#.#L#L#L#L#########.#L#L##.########.#####.#L###.###L#L#L#
LLLL#.LLLLL####.###########..L##LL.L.L.LLLL#########.LLLL##.##############.LL######.LLLLLL
#L#LL.#L#LLL########.###########L#L#L#L##.##########.#L#L##.###.##########L#L#.####..LL#L#
LLLL#L.LLL#L#.#######LLLLLL.####LLLL.LLLLLL.########.LLLL##.########.##.##..L######.#LLLLL
#L#L#.#L#L..#####.##.L#L#L#.####L#.#L#L#L#L.L#######L#L#L###########.#####.#L##.##L.LL##L#
....L........#..#...........#.#............#.....##..L.L.#.........######.LL..#..#.#..#...
#L.############.###########.####L#.#L#L#L#L.LL#L####.#L###################.#..####..L###L#
LL###.#########.####.###########.L.LLLLLLL#.#LLL####.LL#.##.########..####.LL##.########LL

#L#L#.#L#L#L#.#L##L#.#.#L#L..L#L#.#L#L#L#L#.###L#L#L.#L#L#L#L#L#L#.#.#L#L#L#L#L#L#L.L#L#L#
LLLLLLLLLLLLLLL.LLLL..LLLLL##LLLLL.LLLLLLLL.L..LLLLLLLLLLLL.LLLL.LLL.LLLLL.LLL.LLLL#LLLLLL
#L#L#.#L#L#L#L############L.LLL#L#.#L#L#L#L#L#.###.#L######.##L#L.L#.#L#L#.#L#L#L#L.L#L#L#
LLLLL.LLLLLLLL#.####.######.##LLLL.LLLLLLLL.L#L###LL.#########L#L#LLL#.LLL.LLLLLLLL.LLLLLL
#L#L#L#L#L#L###.##################.####L#L#.LLL#####.######.LLLLLLL#L.L#L#L#L#L#L##.##L#L#
#L#L#LLLLLLL###.####.##################LLLL.#.######.######..#L#L#L..#LLLLLLLLLLL#L.LLLLLL
.L..L.#.#.##..##.........#....L.L..##.####.L..#.####.#.#..#.......L#...##..#L.#..#.#.#L..#
#L#.#.LLL.#####..#L#.#L#L##.#L.#.#.########.########.######.#L#L#LLL.#####.LL#####L..LL#LL
#L.L..#L########LLLL.LLLL##.LLLLLL.#################.######.LLLL.L########.#L#########LLLL
LL#.#LL.####.##.####.#############.#################.###############.#####LL.#########L#L#
LLLLLL#LL.#####.###########.##################################.##############.#####.##.LLL

#L#L#.#L#L#L#.#L##L#.#.#L#L..L#L#.#L#L#L#L#.###L#L#L.#L#L#L#L#L#L#.#.#L#L#L#L#L#L#L.L#L#L#
LLLLLLLLLLLLLLL.LLLL..LLLLL##LLLLL.LLLLLLLL.L..LLLLLLLLLLLL.LLLL.LLL.LLLLL.LLL.LLLL#LLLLLL
#L#L#.#L#L#L#L#L#L#L#L#LLLL.LLL#L#.#L#L#L#L#L#.#L#.#L#L#L#L.L#L#L.L#.#L#L#.#L#L#L#L.L#L#L#
LLLLL.LLLLLLLLL.LLLL.LLLLLL.LLLLLL.LLLLLLLL.L#LLLLLL.LLLL#L#L#L#L#LLL#.LLL.LLLLLLLL.LLLLLL
#L#L#L#L#L#L#L#.#L#L#LLLLLLLLLLL#L.L#L#L#L#.LLLL##L#.#L#LLL.LLLLLLL#L.L#L#L#L#L#L##.##L#L#
#L#L#LLLLLLLLL#.#L#L.LLLLLLLLL#L#L#LLLLLLLL.#.#LLLLL.LLLLL#..#L#L#L..#LLLLLLLLLLL#L.LLLLLL
.L..L.#.#.#L..LL.........L....L.L..L#.#LLL.L..L.L#L#.#.#..L.......L#...LL..#L.#..#.#.#L..#
#L#.#.LLL.LL#L#..#L#.#L#LLL.#L.#.#.LLLLLLLL.LLL#LLLL.LLLLL#.#L#L#LLL.LLLLL.LLLLLLLL..LL#LL
#L.L..#L#L#LLLLLLLLL.LLLLLL.LLLLLL.#LLLLLLLLLLLLL#L#.#L#LLL.LLLL.LLLLLLLLL.#L#L#L#L#L#LLLL
LL#.#LL.#L#L.##.#LLL.LLLLLLLLLLLLL.LLLLLLLLLLLLLLLLL.LLLLLLLLLLLLLLL.LLLLLLL.LLLLLLLLLL#L#
LLLLLL#LL.LLLLL.LLLLLLLLLLL.LLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL.LLLLLLLLLLLLLL.#L#LL.LL.LLL

#L#L#.#L#L#L#.#L##L#.#.#L#L..L#L#.#L#L#L#L#.###L#L#L.#L#L#L#L#L#L#.#.#L#L#L#L#L#L#L.L#L#L#
LLLLLLLLLLLLLLL.LLLL..LLLLL##LLLLL.LLLLLLLL.L..LLLLLLLLLLLL.LLLL.LLL.LLLLL.LLL.LLLL#LLLLLL
#L#L#.#L#L#L#L#L#L#L#L#L#LL.LLL#L#.#L#L#L#L#L#.#L#.#L#L#L#L.L#L#L.L#.#L#L#.#L#L#L#L.L#L#L#
LLLLL.LLLLLLLLL.LLLL.LLLLL#.##LLLL.LLLLLLLL.L#LLLLLL.LLLL#L#L#L#L#LLL#.LLL.LLLLLLLL.LLLLLL
#L#L#L#L#L#L#L#.#L#L#L#L#LLLLLLL#L.L#L#L#L#.LLLL##L#.#L#LLL.LLLLLLL#L.L#L#L#L#L#L##.##L#L#
#L#L#LLLLLLLLL#.#L#L.L#LLL#L#L#L#L#LLLLLLLL.#.#LLLLL.LLLLL#..#L#L#L..#LLLLLLLLLLL#L.LLLLLL
.L..L.#.#.#L..LL.........#....L.L..L#.#L#L.L..L.L#L#.#.#..L.......L#...LL..#L.#..#.#.#L..#
#L#.#.LLL.LL#L#..#L#.#L#LLL.#L.#.#.LLLLLLL#.L#L#LLLL.LLLLL#.#L#L#LLL.LLLL#.LLLLLLLL..LL#LL
#L.L..#L#L#LLLLLLLLL.LLLLLL.LLLLLL.#L#L#LLLLLLLLL#L#.#L#LLL.LLLL.LLLLLLLLL.#L#L#L#L#L#LLLL
LL#.#LL.#L#L.##.#L#L.L#L#LLLLLLLLL.LLLLLLLLLLLLLLLLL.LLLLLLLLLLLLLLL.LL#L#LL.LLLLLLLLLL#L#
LLLLLL#LL.LLLLL.LLLL#LLLLLL.LLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL.LLLLLLLLLLLLL#.#L#LL.LL.LLL

...

#L#L#.#L#L#L#.#L##L#.#.#L#L..L#L#.#L#L#L#L#.###L#L#L.#L#L#L#L#L#L#.#.#L#L#L#L#L#L#L.L#L#L#
LLLLLLLLLLLLLLL.LLLL..LLLLL##LLLLL.LLLLLLLL.L..LLLLLLLLLLLL.LLLL.LLL.LLLLL.LLL.LLLL#LLLLLL
#L#L#.#L#L#L#L#L#L#L#L#L#LL.LLL#L#.#L#L#L#L#L#.#L#.#L#L#L#L.L#L#L.L#.#L#L#.#L#L#L#L.L#L#L#
LLLLL.LLLLLLLLL.LLLL.LLLLL#.##LLLL.LLLLLLLL.L#LLLLLL.LLLL#L#L#L#L#LLL#.LLL.LLLLLLLL.LLLLLL
#L#L#L#L#L#L#L#.#L#L#L#L#LLLLLLL#L.L#L#L#L#.LLLL##L#.#L#LLL.LLLLLLL#L.L#L#L#L#L#L##.##L#L#
#L#L#LLLLLLLLL#.#L#L.L#LLL#L#L#L#L#LLLLLLLL.#.#LLLLL.LLLLL#..#L#L#L..#LLLLLLLLLLL#L.LLLLLL
.L..L.#.#.#L..LL.........#....L.L..L#.#L#L.L..L.L#L#.#.#..L.......L#...#L..#L.#..#.#.#L..#
#L#.#.LLL.LL#L#..#L#.#L#LLL.#L.#.#.LLLLLLL#.L#L#LLLL.LLLLL#.#L#L#LLL.#L#L#.LLLLLLLL..LL#LL
#L.L..#L#L#LLLLLLLLL.LLLLL#.LLLLLL.#L#L#LLLLLLLLL#L#.#L#LLL.LLLL.LL#LLLLLL.#L#L#L#L#L#LLLL
LL#.#LL.#L#L.##.#L#L.L#L#L#L##L#L#.LLLLLL#L#L#L#LLLL.LLLL#L#L#L#L#LL.#L#L#LL.LLLLLLLLLL#L#
LLLLLL#LL.LLLLL.LLLL#LLLLLL.LLLLLL####L#LLLLLLLLL##L#L#LLLLLLL.LLLLLLLLLLLLL#.#L#LL.LL.LLL

#L#L#.#L#L#L#.#L##L#.#.#L#L..L#L#.#L#L#L#L#.###L#L#L.#L#L#L#L#L#L#.#.#L#L#L#L#L#L#L.L#L#L#
LLLLLLLLLLLLLLL.LLLL..LLLLL##LLLLL.LLLLLLLL.L..LLLLLLLLLLLL.LLLL.LLL.LLLLL.LLL.LLLL#LLLLLL
#L#L#.#L#L#L#L#L#L#L#L#L#LL.LLL#L#.#L#L#L#L#L#.#L#.#L#L#L#L.L#L#L.L#.#L#L#.#L#L#L#L.L#L#L#
LLLLL.LLLLLLLLL.LLLL.LLLLL#.##LLLL.LLLLLLLL.L#LLLLLL.LLLL#L#L#L#L#LLL#.LLL.LLLLLLLL.LLLLLL
#L#L#L#L#L#L#L#.#L#L#L#L#LLLLLLL#L.L#L#L#L#.LLLL##L#.#L#LLL.LLLLLLL#L.L#L#L#L#L#L##.##L#L#
#L#L#LLLLLLLLL#.#L#L.L#LLL#L#L#L#L#LLLLLLLL.#.#LLLLL.LLLLL#..#L#L#L..#LLLLLLLLLLL#L.LLLLLL
.L..L.#.#.#L..LL.........#....L.L..L#.#L#L.L..L.L#L#.#.#..L.......L#...#L..#L.#..#.#.#L..#
#L#.#.LLL.LL#L#..#L#.#L#LLL.#L.#.#.LLLLLLL#.L#L#LLLL.LLLLL#.#L#L#LLL.#L#L#.LLLLLLLL..LL#LL
#L.L..#L#L#LLLLLLLLL.LLLLL#.LLLLLL.#L#L#LLLLLLLLL#L#.#L#LLL.LLLL.LL#LLLLLL.#L#L#L#L#L#LLLL
LL#.#LL.#L#L.##.#L#L.L#L#L#L##L#L#.LLLLLL#L#L#L#LLLL.LLLL#L#L#L#L#LL.#L#L#LL.LLLLLLLLLL#L#
LLLLLL#LL.LLLLL.LLLL#LLLLLL.LLLLLL####L#LLLLLLLLL##L#L#LLLLLLL.LLLLLLLLLLLLL#.#L#LL.LL.LLL

#L#L#.#L#L#L#.#L##L#.#.#L#L..L#L#.#L#L#L#L#.###L#L#L.#L#L#L#L#L#L#.#.#L#L#L#L#L#L#L.L#L#L#
LLLLLLLLLLLLLLL.LLLL..LLLLL##LLLLL.LLLLLLLL.L..LLLLLLLLLLLL.LLLL.LLL.LLLLL.LLL.LLLL#LLLLLL
#L#L#.#L#L#L#L#L#L#L#L#L#LL.LLL#L#.#L#L#L#L#L#.#L#.#L#L#L#L.L#L#L.L#.#L#L#.#L#L#L#L.L#L#L#
LLLLL.LLLLLLLLL.LLLL.LLLLL#.##LLLL.LLLLLLLL.L#LLLLLL.LLLL#L#L#L#L#LLL#.LLL.LLLLLLLL.LLLLLL
#L#L#L#L#L#L#L#.#L#L#L#L#LLLLLLL#L.L#L#L#L#.LLLL##L#.#L#LLL.LLLLLLL#L.L#L#L#L#L#L##.##L#L#
#L#L#LLLLLLLLL#.#L#L.L#LLL#L#L#L#L#LLLLLLLL.#.#LLLLL.LLLLL#..#L#L#L..#LLLLLLLLLLL#L.LLLLLL
.L..L.#.#.#L..LL.........#....L.L..L#.#L#L.L..L.L#L#.#.#..L.......L#...#L..#L.#..#.#.#L..#
#L#.#.LLL.LL#L#..#L#.#L#LLL.#L.#.#.LLLLLLL#.L#L#LLLL.LLLLL#.#L#L#LLL.#L#L#.LLLLLLLL..LL#LL
#L.L..#L#L#LLLLLLLLL.LLLLL#.LLLLLL.#L#L#LLLLLLLLL#L#.#L#LLL.LLLL.LL#LLLLLL.#L#L#L#L#L#LLLL
LL#.#LL.#L#L.##.#L#L.L#L#L#L##L#L#.LLLLLL#L#L#L#LLLL.LLLL#L#L#L#L#LL.#L#L#LL.LLLLLLLLLL#L#
LLLLLL#LL.LLLLL.LLLL#LLLLLL.LLLLLL####L#LLLLLLLLL##L#L#LLLLLLL.LLLLLLLLLLLLL#.#L#LL.LL.LLL

#L#L#.#L#L#L#.#L##L#.#.#L#L..L#L#.#L#L#L#L#.###L#L#L.#L#L#L#L#L#L#.#.#L#L#L#L#L#L#L.L#L#L#
LLLLLLLLLLLLLLL.LLLL..LLLLL##LLLLL.LLLLLLLL.L..LLLLLLLLLLLL.LLLL.LLL.LLLLL.LLL.LLLL#LLLLLL
#L#L#.#L#L#L#L#L#L#L#L#L#LL.LLL#L#.#L#L#L#L#L#.#L#.#L#L#L#L.L#L#L.L#.#L#L#.#L#L#L#L.L#L#L#
LLLLL.LLLLLLLLL.LLLL.LLLLL#.##LLLL.LLLLLLLL.L#LLLLLL.LLLL#L#L#L#L#LLL#.LLL.LLLLLLLL.LLLLLL
#L#L#L#L#L#L#L#.#L#L#L#L#LLLLLLL#L.L#L#L#L#.LLLL##L#.#L#LLL.LLLLLLL#L.L#L#L#L#L#L##.##L#L#
#L#L#LLLLLLLLL#.#L#L.L#LLL#L#L#L#L#LLLLLLLL.#.#LLLLL.LLLLL#..#L#L#L..#LLLLLLLLLLL#L.LLLLLL
.L..L.#.#.#L..LL.........#....L.L..L#.#L#L.L..L.L#L#.#.#..L.......L#...#L..#L.#..#.#.#L..#
#L#.#.LLL.LL#L#..#L#.#L#LLL.#L.#.#.LLLLLLL#.L#L#LLLL.LLLLL#.#L#L#LLL.#L#L#.LLLLLLLL..LL#LL
#L.L..#L#L#LLLLLLLLL.LLLLL#.LLLLLL.#L#L#LLLLLLLLL#L#.#L#LLL.LLLL.LL#LLLLLL.#L#L#L#L#L#LLLL
LL#.#LL.#L#L.##.#L#L.L#L#L#L##L#L#.LLLLLL#L#L#L#LLLL.LLLL#L#L#L#L#LL.#L#L#LL.LLLLLLLLLL#L#
LLLLLL#LL.LLLLL.LLLL#LLLLLL.LLLLLL####L#LLLLLLLLL##L#L#LLLLLLL.LLLLLLLLLLLLL#.#L#LL.LL.LLL

#L#L#.#L#L#L#.#L##L#.#.#L#L..L#L#.#L#L#L#L#.###L#L#L.#L#L#L#L#L#L#.#.#L#L#L#L#L#L#L.L#L#L#
LLLLLLLLLLLLLLL.LLLL..LLLLL##LLLLL.LLLLLLLL.L..LLLLLLLLLLLL.LLLL.LLL.LLLLL.LLL.LLLL#LLLLLL
#L#L#.#L#L#L#L#L#L#L#L#L#LL.LLL#L#.#L#L#L#L#L#.#L#.#L#L#L#L.L#L#L.L#.#L#L#.#L#L#L#L.L#L#L#
LLLLL.LLLLLLLLL.LLLL.LLLLL#.##LLLL.LLLLLLLL.L#LLLLLL.LLLL#L#L#L#L#LLL#.LLL.LLLLLLLL.LLLLLL
#L#L#L#L#L#L#L#.#L#L#L#L#LLLLLLL#L.L#L#L#L#.LLLL##L#.#L#LLL.LLLLLLL#L.L#L#L#L#L#L##.##L#L#
#L#L#LLLLLLLLL#.#L#L.L#LLL#L#L#L#L#LLLLLLLL.#.#LLLLL.LLLLL#..#L#L#L..#LLLLLLLLLLL#L.LLLLLL
.L..L.#.#.#L..LL.........#....L.L..L#.#L#L.L..L.L#L#.#.#..L.......L#...#L..#L.#..#.#.#L..#
#L#.#.LLL.LL#L#..#L#.#L#LLL.#L.#.#.LLLLLLL#.L#L#LLLL.LLLLL#.#L#L#LLL.#L#L#.LLLLLLLL..LL#LL
#L.L..#L#L#LLLLLLLLL.LLLLL#.LLLLLL.#L#L#LLLLLLLLL#L#.#L#LLL.LLLL.LL#LLLLLL.#L#L#L#L#L#LLLL
LL#.#LL.#L#L.##.#L#L.L#L#L#L##L#L#.LLLLLL#L#L#L#LLLL.LLLL#L#L#L#L#LL.#L#L#LL.LLLLLLLLLL#L#
LLLLLL#LL.LLLLL.LLLL#LLLLLL.LLLLLL####L#LLLLLLLLL##L#L#LLLLLLL.LLLLLLLLLLLLL#.#L#LL.LL.LLL

# --- Part Two ---

As soon as people start to arrive, you realize your mistake. People don't just care about adjacent seats - they care about the first seat they can see in each of those eight directions!

Now, instead of considering just the eight immediately adjacent seats, consider the first seat in each of those eight directions. For example, the empty seat below would see eight occupied seats:

    .......#.
    ...#.....
    .#.......
    .........
    ..#L....#
    ....#....
    .........
    #........
    ...#.....

The leftmost empty seat below would only see one empty seat, but cannot see any of the occupied ones:

    .............
    .L.L.#.#.#.#.
    .............

The empty seat below would see no occupied seats:

    .##.##.
    #.#.#.#
    ##...##
    ...L...
    ##...##
    #.#.#.#
    .##.##.

Also, people seem to be more tolerant than you expected: it now takes five or more visible occupied seats for an occupied seat to become empty (rather than four or more from the previous rules). The other rules still apply: empty seats that see no occupied seats become occupied, seats matching no rule don't change, and floor never changes.

Given the same starting layout as above, these new rules cause the seating area to shift around as follows:

    L.LL.LL.LL
    LLLLLLL.LL
    L.L.L..L..
    LLLL.LL.LL
    L.LL.LL.LL
    L.LLLLL.LL
    ..L.L.....
    LLLLLLLLLL
    L.LLLLLL.L
    L.LLLLL.LL

    #.##.##.##
    #######.##
    #.#.#..#..
    ####.##.##
    #.##.##.##
    #.#####.##
    ..#.#.....
    ##########
    #.######.#
    #.#####.##

    #.LL.LL.L#
    #LLLLLL.LL
    L.L.L..L..
    LLLL.LL.LL
    L.LL.LL.LL
    L.LLLLL.LL
    ..L.L.....
    LLLLLLLLL#
    #.LLLLLL.L
    #.LLLLL.L#

    #.L#.##.L#
    #L#####.LL
    L.#.#..#..
    ##L#.##.##
    #.##.#L.##
    #.#####.#L
    ..#.#.....
    LLL####LL#
    #.L#####.L
    #.L####.L#

    #.L#.L#.L#
    #LLLLLL.LL
    L.L.L..#..
    ##LL.LL.L#
    L.LL.LL.L#
    #.LLLLL.LL
    ..L.L.....
    LLLLLLLLL#
    #.LLLLL#.L
    #.L#LL#.L#

    #.L#.L#.L#
    #LLLLLL.LL
    L.L.L..#..
    ##L#.#L.L#
    L.L#.#L.L#
    #.L####.LL
    ..#.#.....
    LLL###LLL#
    #.LLLLL#.L
    #.L#LL#.L#

    #.L#.L#.L#
    #LLLLLL.LL
    L.L.L..#..
    ##L#.#L.L#
    L.L#.LL.L#
    #.LLLL#.LL
    ..#.L.....
    LLL###LLL#
    #.LLLLL#.L
    #.L#LL#.L#

Again, at this point, people stop shifting around and the seating area reaches equilibrium. Once this occurs, you count 26 occupied seats.

Given the new visibility method and the rule change for occupied seats becoming empty, once equilibrium is reached, how many seats end up occupied?


In [67]:
import numpy as np

with open("day11_test.txt") as f:
    seats = f.readlines()
seats_current = [line.strip() for line in seats]

def count_visible(row, col, seats, show_used=False):
    seat_count = 0
    row_seats = ''
    col_seats = ''
    forward_slash = ''
    back_slash = ''
    for row_num, seat_rows in enumerate(seats):
#         print(seat_rows)
        for col_num, seat in enumerate(seat_rows):
            if row_num == row:
                row_seats += seat
            if col_num == col:
                col_seats += seat
            back_slash_val = (row_num - row) * col_num + col
            front_slash_val = -(row_num - row) * col_num + col
            if back_slash_val == 0:
                back_slash += seat
            if front_slash_val == 0:
                forward_slash += seat
    print(f"row: {row_seats}")
    print(f"col: {col_seats}")
    print(f"front: {back_slash}")
    print(f"back: {forward_slash}")
    print()
    return seat_count

# moving = True
# while moving:
#     seats_future = []
#     for row_num, seat_rows in enumerate(seats_current):
#         new_seat_row = ''
#         for col_num, seat in enumerate(seat_rows):
#             if seat == "#" or seat == "L":
#                 num_occupied = count_visible(row_num, col_num, seats_current)
                
#                 if seat == "L":
#                     if num_occupied == 0:
#                         new_seat_row += "#"
#                     else:
#                         new_seat_row += seat
#                 if seat == "#":
#                     if num_occupied >= 5:
#                         new_seat_row += "L"
#                     else:
#                         new_seat_row += seat

#             else:
#                 new_seat_row += seat
#         seats_future.append(new_seat_row)
    
# #     zero_two = count_occupied_adjacent(2, 0, seats_current, show_used=True)
# #     print(f"zero, two: {zero_two}")
    
#     [print(row) for row in seats_current]
#     print()
#     if seats_future == seats_current:
#         moving = False
#     else:
#         seats_current = seats_future.copy()

# full_count = 0
# for seat_rows in seats_current:
#     for seat in seat_rows:
#         if seat == "#":
#             full_count += 1
# print(full_count)

zero_two = count_visible(2, 0, seats_current, show_used=True)
#     print(f"zero, two: {zero_two}")

row: L.L.L..L..
col: LLLLLL.LLL
front: LLL.L.L..L..LLL.LLL
back: LLL.L.L..L..LLL.LLL

